# 3. Guide - 1. Keras - 5. Save and serialize models

목차
1. Setup
2. Part 1: Saving Sequential models or Functional models
 1. Whole-model saving
 2. Export to SavedModel
 3. Architecture-only saving
 4. Weights-only saving
 5. Weights=only saving in SavedModel format
3. Saving Subclassed Models

## 1. Setup

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
tf.keras.backend.clear_session()

## 2. Part 1: Saving Sequential models or Functional models

In [2]:
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(784,), name='digits')
x = layers.Dense(64, activation='relu', name='dense_1')(inputs)
x = layers.Dense(64, activation='relu', name='dense_2')(x)
outputs = layers.Dense(10, activation='softmax', name='predictions')(x)

model = keras.Model(inputs=inputs, outputs=outputs, name='3_layer_mlp')
model.summary()

Model: "3_layer_mlp"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
digits (InputLayer)          [(None, 784)]             0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                50240     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
predictions (Dense)          (None, 10)                650       
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________


In [3]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=keras.optimizers.RMSprop())
history = model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=1)

60000/60000 [==============================] - 3s 42us/sample - loss: 0.3150


In [4]:
predictions = model.predict(x_test)

### 1. Whole-model saving

In [5]:
model.save('path_to_my_model.h5')

new_model = keras.models.load_model('path_to_my_model.h5')

In [6]:
import numpy as np

new_predictions = new_model.predict(x_test)
np.testing.assert_allclose(predictions, new_predictions, atol=1e-6)

### 2. Export to SavedModel

In [7]:
keras.experimental.export_saved_model(model, 'path_to_saved_model')

new_model = keras.experimental.load_from_saved_model('path_to_saved_model')

new_predictions = new_model.predict(x_test)
np.testing.assert_allclose(predictions, new_predictions, atol=1e-6)

W0522 12:44:59.591393 18308 deprecation.py:323] From C:\Users\la\Anaconda3\envs\tf20a\lib\site-packages\tensorflow\python\saved_model\signature_def_utils_impl.py:253: build_tensor_info (from tensorflow.python.saved_model.utils_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
W0522 12:44:59.592390 18308 tf_logging.py:161] Export includes no default signature!
W0522 12:45:00.048781 18308 tf_logging.py:161] Export includes no default signature!


### 3. Architecture-only saving

In [8]:
config = model.get_config()
reinitialized_model = keras.Model.from_config(config)

new_predictions = reinitialized_model.predict(x_test)
assert abs(np.sum(predictions - new_predictions)) > 0

In [9]:
json_config = model.to_json()
reinitialized_model = keras.models.model_from_json(json_config)

### 4. Weights-only saving

In [10]:
weights = model.get_weights()
model.set_weights(weights)

In [11]:
config = model.get_config()
weights = model.get_weights()

new_model = keras.Model.from_config(config)
new_model.set_weights(weights)

new_predictions = new_model.predict(x_test)
np.testing.assert_allclose(predictions, new_predictions, atol=1e-6)

In [12]:
json_config = model.to_json()
with open('model_config.json','w') as json_file:
    json_file.write(json_config)
model.save_weights('path_to_my_weights.h5')

with open('model_config.json') as json_file:
    json_config = json_file.read()
new_model = keras.models.model_from_json(json_config)
new_model.load_weights('path_to_my_weights.h5')

new_predictions = new_model.predict(x_test)
np.testing.assert_allclose(predictions, new_predictions, atol=1e-6)

In [13]:
model.save('path_to_my_model.h5')
del model
model = keras.models.load_model('path_to_my_model.h5')

### 5. Weights=only saving in SavedModel format

In [14]:
model.save_weights('path_to_my_tf_savedmodel')
model.save_weights('path_to_my_tf_savedmodel', save_format = 'tf')

## 3. Saving Subclassed Models

In [15]:
class ThreeLayerMLP(keras.Model):
    
    def __init__(self, name=None):
        super(ThreeLayerMLP, self).__init__(name=name)
        self.dense_1 = layers.Dense(64, activation='relu', name='dense_1')
        self.dense_2 = layers.Dense(64, activation='relu', name='dense_2')
        self.pred_layer = layers.Dense(10, activation='softmax', name='predictions')
        
    def call(self, inputs):
        x = self.dense_1(inputs)
        x = self.dense_2(x)
        return self.pred_layer(x)
    
def get_model():
    return ThreeLayerMLP(name='3_layer_mlp')

model = get_model()

In [16]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=keras.optimizers.RMSprop())
history = model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=1)

60000/60000 [==============================] - 2s 35us/sample - loss: 0.3129


In [17]:
model.save_weights('path_to_my_weights', save_format='tf')

In [18]:
predictions = model.predict(x_test)
first_batch_loss = model.train_on_batch(x_train[:64], y_train[:64])

In [19]:
new_model = get_model()
new_model.compile(loss='sparse_categorical_crossentropy',
                 optimizer = keras.optimizers.RMSprop())

new_model.train_on_batch(x_train[:1], y_train[:1])

new_model.load_weights('path_to_my_weights')
new_predictions = new_model.predict(x_test)
np.testing.assert_allclose(predictions, new_predictions, atol=1e-6)

new_first_batch_loss = new_model.train_on_batch(x_train[:64], y_train[:64])
assert first_batch_loss == new_first_batch_loss